# asf_search - Basic Overview
`asf_search` provides several helpful classes and functions to simplify working with the ASF catalog. This notebook briefly describes how to get started with `asf_search`.
***
## Before You Start
  
The steps outlined in this notebook assume `asf_search` is available on your system. `asf_search` is available through [PyPi](https://pypi.org/project/asf-search/), [Conda](https://anaconda.org/conda-forge/asf_search), and [Github](https://github.com/asfadmin/Discovery-asf_search). Additionally, full documentation for `asf_search` and many other services offered by ASF is available at [https://docs.asf.alaska.edu/](https://docs.asf.alaska.edu/)
  
For this demonstration, we have already installed `asf_search` within a virtual environment through PyPi via the command:
  
  
```pip install asf_search```
  
`asf_search` requires Python 3.6 or higher.

***
## Usage
Once installed, simply import `asf_search` as you would any other Python module:

In [ ]:
import asf_search as asf

`asf_search` version numbers are based on [Semantic Versioning 2.0.0](https://semver.org/spec/v2.0.0.html):

In [ ]:
asf.__version__

`asf_search` automatically reports search errors to ASF. To opt out, simply add the following line after importing the package:

In [ ]:
asf.REPORT_ERRORS = False

If you have any questions regarding automatic error reporting, email uso@asf.alaska.edu

***

## Performing a Basic Search
[View this search in Vertex](https://search.asf.alaska.edu/#/?resultsLoaded=true&dataset=SENTINEL-1)

For this basic example, we will specify two search parameters: a platform, and how many results we want to retrieve:
- Sentinel-1
- 5 results max
    - (results are retrieved newest-first, so this will be the 5 newest products).

In [ ]:
results = asf.search(platform=asf.PLATFORM.SENTINEL1, maxResults=5)

Alternatively, it may be useful to handle your search arguments as a dictionary:

In [ ]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'maxResults': 5
}
results = asf.search(**opts)

If `search()` encounters an error while querying CMR the search will halt, log the error, and return all results up until the error occurred. To check that results are complete programmatically use `ASFSearchResults.raise_if_incomplete()` in a `try-except` statement:

In [ ]:
from asf_search.exceptions import ASFSearchError

try:
    results.raise_if_incomplete()
except ASFSearchError as e:
    pass #handle the error here

`search()` is itself a convenient wrapper around `search_generator()`, which queries CMR and yields `ASFSearchResults` page-by-page. If `search_generator()` completed without encountering an error, the last page will mark `ASFSearchResults.searchComplete` to `True`. Check the final page's `searchComplete` flag to assert results status or mark the final `ASFSearchResults` object as complete.

In [ ]:
paged_results = asf.ASFSearchResults([])

pages = asf.search_generator(**opts)

for page in pages:
    paged_results.extend(page)

    # The last page will be marked as complete once all results have returned successfully,
    # keep track by updating ASFSearchResults manually if you wish to use raise_if_incomplete()    
    paged_results.searchComplete = page.searchComplete
    paged_results.searchOptions = page.searchOptions


try:
    paged_results.raise_if_incomplete()
except ASFSearchError as e:
    pass #handle the error here

Note the use of `asf_search`-provided constants, as many editors support use of these through autocompletion. Categories of constants include:
- `BEAMMODE`
- `FLIGHT_DIRECTION`
- `INSTRUMENT`
- `PLATFORM`
- `POLARIZATION`
- `PRODUCT_TYPE`
  


`asf_search` also provides the `search_count()` method, which will return the count of total results matching the passed search options. 
For example passing the same `opts` object we used for our search above will return the current size of the `SENTINEL1` catalog.

In [ ]:
count = asf.search_count(**opts)

***
## Working With Results
### `ASFSearchResults`
Search results are returned as an `ASFSearchResults` object, a subclass of `UserList`, containing a list of `ASFProduct` objects, each of these classes providing some additional functionality to aid in working with the results and individual products:

In [ ]:
results

### `ASFProduct`
`ASFProduct` provides a number of metadata fields, such as:
- Geographic coordinates
    - Latitude/Longitude
    - Shape type
- Scene and product metadata
    - Path, frame
    - Platform, beam, polarization
    - File name, size, URL
    - and many others
  
Geographic coordinates are stored in the `geometry` attribute:

In [ ]:
results[0].geometry

Other metadata is available through the `properties` attribute:

In [ ]:
results[0].properties

### Output
The layout of the above data structure mirrors the geojson output format provided by ASF's SearchAPI, for a smooth transition to `asf_search`.
  
In fact, the `ASFSearchResults.__str__()` method serializes results to geojson _identical_ to that of ASF's SearchAPI, allowing for a drop-in replacement for users of the SearchAPI, simply by explicitly or implicitly casting search results to a string:

In [ ]:
print(results)

Additionally, individual `ASFProduct` objects provides geojson-based serialization, in the form of a geojson `Feature` snippet:

In [ ]:
print(results[0])

`ASFSearchResults` also supports the following output formats:
- `csv`
- `jsonlite`
- `jsonlite2`
- `metalink`
- `kml`

All formats are callable as methods on any `ASFSearchResults` object, producing results equivalent to SearchAPI.

Below is an example of saving results to a csv file using the `ASFSearchResults.csv()` method.

In [ ]:
with open("search_results.csv", "w") as f:
   f.writelines(results.csv())

The output can be also previewed in the terminal

In [ ]:
print(*results.csv(), sep='')

For more advanced usage, different output formats can be used with the `asf_search.search_generator()`, and can be used to stream results as they are returned from CMR, page-by-page

In [ ]:
# asf-search queries CMR with page sizes of 500, 
# so setting maxResults=1000 means asf-search will have to query cmr twice, each time returning 500 products
large_results_generator = asf.search_generator(maxResults=1000, platform=asf.SENTINEL1A)

with open("search_results.metalink", "w") as f:
    f.writelines(asf.export.results_to_metalink(large_results_generator))

## Summary
A complete example, showing how simple use of `asf_search` can be:

In [ ]:
import asf_search as asf

print(asf.search(platform='S1', maxResults=5))

***
Next: [Geographic Searches](./2-Geographic_Search.ipynb)